In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torchvision import models
from torchvision import transforms as tf
import torch.nn. functional as F


In [ ]:
alexnet = models.alexnet (pretrained-True) . features
for param in alexnet.parameters():
    param. requires_grad_(False)

In [ ]:
device = torch.device ("cpu")
if torch. cuda.is_available():
    device torch.device("cuda")
alexnet.to (device)

In [ ]:
mean= (0.485, 0.456, 0.406)
std (0.229, 0.224, 0.225)

In [ ]:
def transformation (img):
    tasks = tf.Compose( [tf.Resize (256),
                         tf.To Tensor(),
                         tf.Normalize (mean, std)])
    img tasks (img)
    img img.unsqueeze (0)
    return img

In [ ]:
content img = Image.open("cat.jpg").convert('RGB')
style_img= Image.open("starry_night.jpg").convert('RGB')

In [ ]:
content_img

In [ ]:
content_img.size

In [ ]:
list(content_img.getdata())

In [ ]:
style_img

In [ ]:
style_img.size

In [ ]:
list(style_img.getdata())

In [ ]:
content_img= transformation (content_img).to (device)
style_img= transformation (style_img).to (device)

In [ ]:
content_img.shape
style_img.shape

In [ ]:
def tensor_to_image (tensor):
    image= tensor.clone ().detach()
    image image.cpu ().numpy ().squeeze()
    image = image.transpose (1, 2, 0)
    image = np.array(std) + np.array(mean)
    image image.clip(0, 1)
    return image

In [ ]:
img= tensor_to_image(style_img)
fig = plt.figure()
fig.suptitle('Style Image')
plt.imshow(img)
img = tensor_to_image(content_img)
fig= plt.figure()
fig.suptitle('Content Image')
plt.imshow(img)

In [ ]:
LAYERS_OF_INTEREST = {'0': 'conv1_1',
'3': 'conv2_1',
'6': 'conv3_1',
'8': 'conv3_2',
'10': 'conv3_3'}

In [ ]:
def apply_model_and_extract_features (image, model):
    x = image
    features = {}
    for name, layer in model._modules.items():
        x = layer (x)
        if name in LAYERS_OF_INTEREST:
            features [LAYERS_OF_INTEREST [name]] = x
    return features

In [ ]:
content_img_features = apply_model_and_extract_features (content_img, vgg)
style_img_features = apply_model_and_extract_features (style_img, vgg)

In [ ]:
def calculate_gram_matrix(tensor):
    _,channels, height, width = tensor.size()

    tensor = tensor.view (channels, height * width)
    gram_matrix = torch.mm (tensor, tensor.t())
    gram_matrix = gram_matrix.div (channels * height* width)
    return gram_matrix

In [ ]:
style_features_gram_matrix = layer: calculate_gram_matrix (style_img_features [layer]) for layer in
style_img_reaturess


In [ ]:
style_features_gram_matrix

In [ ]:
weights = {'conv1_1': 1.0, 'conv2_1': 0.8, 'conv3_1': 0.25}

In [ ]:
target= content_img.clone (). requires_grad_(True).to (device)
optimizer optim. RMSprop( [target], lr=0.003)

In [ ]:
for i in range (1, 2000):
    target_features = apply_model_and_extract_features (
    content_loss F.mse_loss (target_features ['conv3_1'], content_img_features ['conv3_1'])    
    style_loss = 0
    for layer in weights:
        target_feature= target_features [layer]
        target gram_matrix = calculate_gram_matrix (target_feature
        style_gram_matrix = style_features_gram_matrix [
        layer_loss = F. mse_loss (target_gram_matrix, style_gram_matrix
        layer_loss weights [layer]
        channels, height, width = target_feature.shape
        style_loss + layer_loss
    total loss = 1000000 style_loss + content_loss
    if i % 50 == 0:
        print ('Epoch {}:, Style Loss {:4f}, Content Loss : {:4f}'.format( i, style_loss, content_loss))   
    optimizer.zero_grad()
    total loss.backward()
    optimizer.step()                              

In [ ]:
fig, (ax1, ax2) = plt. subplots (1, 2, figsize=(20, 10)
ax1.imshow (tensor_to_image (content_img))
ax2.imshow (tensor_to_image (target))